In [1]:
import logging
from django.utils.timezone import utc
from bs4 import BeautifulSoup
from datetime import datetime, timedelta, date
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from datetime import datetime
import time
from enum import Enum, IntEnum
from collections import Counter
import os
import csv
import pandas as pd
import traceback
import sys
from typing import List, Union
import logging
import html5lib




/tmp/ipykernel_234/2233749002.py:2: RemovedInDjango50Warning: The django.utils.timezone.utc alias is deprecated. Please update your code to use datetime.timezone.utc instead.
  from django.utils.timezone import utc


In [32]:
driver = webdriver.Chrome()
driver.set_window_size(1400,1000)
driver.get("https://www.pepper.pl/nowe")
time.sleep(0.7)
page = driver.page_source
soup = BeautifulSoup(page, "html5lib")


articles = soup.find_all('article')

for article in articles:
    try:
        name = article.find_all(attrs={'class': "cept-tt thread-link linkPlain thread-title--list js-thread-title"})[0]['title']
    except:
        name = 'name_failed'
    try:
        item_id = int(article.get("id").strip('thread_'))
        item_id = item_id.strip('thread_')
        item_id = int(item_id)
    except:
        item_id = 'id_failed'
    try:
        discount_price = article.find_all(attrs={'class': "thread-price text--b cept-tp size--all-l size--fromW3-xl"})
        discount_price = float(discount_price[0].get_text().strip('zł').replace('.','').replace(',','.'))
    except:
        discount_price = 'discount_pr_failed'
    try:
        regular_price = article.find_all(attrs={'class': "mute--text text--lineThrough size--all-l size--fromW3-xl"})
        regular_price = float(regular_price[0].get_text().strip('zł').replace('.','').replace(',','.'))
    except:
        regular_price = 'reg_pr_failed'    
    try:
        percentage_discount = article.find_all(attrs={'class': "space--ml-1 size--all-l size--fromW3-xl"})
        percentage_discount = float(percentage_discount[0].get_text().strip('%'))
    except:
        percentage_discount = 'per_dis_failed'
    try:
        item_url = article.find_all('a', {"class":"cept-tt thread-link linkPlain thread-title--list js-thread-title"})[0]['href']
    except:
        item_url = 'url_failed'
    try:
        date_tag = article.find_all('div', {"class":"size--all-s flex boxAlign-jc--all-fe boxAlign-ai--all-c flex--grow-1 overflow--hidden"})
    except:
        date_tag = 'date_failed'

    print(name, discount_price, item_id, discount_price, regular_price, percentage_discount, item_url, date_tag)

Laptop ACER Nitro 5 Ryzen 5-6600H/16GB/512GB/RTX3060/17,3'' FHD 4499.0 704540 4499.0 reg_pr_failed per_dis_failed https://www.pepper.pl/promocje/acer-nitro-5-ryzen-516gb512gbrtx3060173-fhd-704540 [<div class="size--all-s flex boxAlign-jc--all-fe boxAlign-ai--all-c flex--grow-1 overflow--hidden"><span class="metaRibbon lbox--v-1 boxAlign-ai--all-c overflow--wrap-off space--l-3 text--color-greyShade"><svg class="icon icon--clock text--color-greyShade space--mr-1" height="22px" width="22px"><use xlink:href="/assets/img/ico_fff5e.svg#clock"></use></svg><span>10 min</span></span></div>]
Biustonosz Gorteks push-up Celine/B1 @Gorteks 61.75 704539 61.75 93.8 -34.0 https://www.pepper.pl/promocje/biustonosz-gorteks-push-up-celineb1-at-gorteks-704539 [<div class="size--all-s flex boxAlign-jc--all-fe boxAlign-ai--all-c flex--grow-1 overflow--hidden"><span class="metaRibbon lbox--v-1 boxAlign-ai--all-c overflow--wrap-off space--l-3 text--color-greyShade"><svg class="icon icon--hourglass text--color

In [34]:
driver = webdriver.Chrome()
driver.set_window_size(1400,1000)
driver.get("https://www.pepper.pl/search?q=s21")
time.sleep(0.7)
page = driver.page_source
soup = BeautifulSoup(page, "html5lib")


articles = soup.find_all('article')

In [37]:
print(type(articles[18]))

<class 'bs4.element.Tag'>


In [8]:
with open("searched_item_found.html", 'w', encoding='utf-8') as file:
    file.write(str(soup))

In [3]:
from pathlib import Path

with open("source/pepper_app/tests/fixtures/to_test_scrap/soup.html", 'r', encoding='utf-8') as file:
    soup = file.read()

soup = BeautifulSoup(soup, 'html5lib')

In [2]:
from pathlib import Path

with open("source/pepper_app/tests/fixtures/to_test_get_info/soup_with_duplicates.html", 'r', encoding='utf-8') as file:
    soup = file.read()

soup = BeautifulSoup(soup, 'html5lib')

In [4]:
a = datetime.utcnow().replace(tzinfo=utc)

print(a)
print(type(a))

2023-08-24 10:38:25.358769+00:00
<class 'datetime.datetime'>


In [4]:
import logging
class CheckConditions:


    def __init__(self, soup: BeautifulSoup) -> None:
        self.soup = soup


    def check_if_last_page(self) -> bool:
        """Checking 'nowe' category to verify if the scraped page is the last one."""
        try:
            searched_ending_string = soup.find_all('h1', {"class":"size--all-xl size--fromW3-xxl text--b space--b-2"})[0].get_text()
            if searched_ending_string.startswith("Ups"):
                logging.warning("No more pages to scrap.")
                return False
        except:
            return True

        """Checking 'search' category to verify if the scraped page is the last one."""
        try:
            searched_ending_string = self.soup.find_all('h3', {"class":"size--all-l"})[0].get_text()
            searched_articles_number = self.soup.find_all('span', {"class":"box--all-i size--all-s vAlign--all-m"})[0].get_text()
            searched_articles_number = int(searched_articles_number.replace(" ","").strip("\n\t Okazje()"))
            if searched_ending_string.startswith("Ups") and searched_articles_number > 0:
                logging.warning("No more pages to scrap.")
                return False
        except:
            return True

    def check_if_no_items_found(self) -> bool:
        """Checking if searched item was found."""
        try:
            searched_ending_string = self.soup.find_all('h3', {"class":"size--all-l"})[0].get_text()
            searched_articles_number = self.soup.find_all('span', {"class":"box--all-i size--all-s vAlign--all-m"})[0].get_text()
            searched_articles_number = int(searched_articles_number.replace(" ","").strip("\n\t Okazje()"))
            if searched_ending_string.startswith("Ups") and searched_articles_number == 0:
                logging.warning("The searched item was not found.")
                return False
        except:
            return True

In [33]:
instance = CheckConditions(soup).check_if_last_page()
print(instance)

False


In [6]:
driver = webdriver.Chrome()
print(type(soup))
print(type(driver))

<class 'bs4.BeautifulSoup'>
<class 'selenium.webdriver.chrome.webdriver.WebDriver'>


In [4]:
articles = soup.find_all('article')


In [16]:
def get_items_details(retrived_articles) -> None:
        start_time = datetime.utcnow().replace(tzinfo=utc)
        all_items = list()
        try:
            for article in retrived_articles:
                item = list()
                item.append(1)
                item.append(2)
                item.append(3)
                item.append(4)
                item.append(5)
                item.append(6)
                item.append(7)
                if item not in all_items:
                    all_items.append(item)
                if '' in item:
                    logging.warning("Data retrieving failed. None values detected")
                    break
                #if to_csv == True:
                    #self.save_data_to_csv(item)
                #if to_database == True:
                    #LoadItemDetailsToDatabase(item).load_to_db()
                return all_items
        except Exception as e:
            logging.warning(f"Getting item details failed :\
                        {e}\n Tracking: {traceback.format_exc()}")

        end_time = datetime.utcnow().replace(tzinfo=utc)
        action_execution_datetime = end_time - start_time
        """
        if to_statistics == True:
            try:
                stats_info = self.get_scraping_stats_info(action_execution_datetime)
                LoadScrapingStatisticsToDatabase(stats_info).load_to_db()
            except Exception as e:
                logging.warning(f"Populating ScrapingStatistics table failed: {e}\n Tracking: {traceback.format_exc()}")
        """

In [16]:
from collections import Counter

articles = soup.find_all('article')


count = Counter(tuple(article) for article in articles)
print(any(val > 1 for val in count.values()))



False


In [12]:
articles = soup.find_all('article')
article = articles[0]
num = 0
for article in articles:

    try:
        name = article.find_all(attrs={'class': "cept-tt thread-link linkPlain thread-title--list js-thread-title"})[0]['title']
    except:
        name = 'name_failed'
    try:
        item_id = article.get("id")
        item_id = item_id.strip('thread_')
        item_id = int(item_id)
    except:
        item_id = 'id_failed'
    try:
        discount_price = article.find_all(attrs={'class': "thread-price text--b cept-tp size--all-l size--fromW3-xl"})
        discount_price = float(discount_price[0].get_text().strip('zł').replace('.','').replace(',','.'))
    except:
        discount_price = 'discount_pr_failed'
    try:
        regular_price = article.find_all(attrs={'class': "mute--text text--lineThrough size--all-l size--fromW3-xl"})
        regular_price = float(regular_price[0].get_text().strip('zł').replace('.','').replace(',','.'))
    except:
        regular_price = 'reg_pr_failed'    
    try:
        percentage_discount = article.find_all(attrs={'class': "space--ml-1 size--all-l size--fromW3-xl"})
        percentage_discount = float(percentage_discount[0].get_text().strip('%'))
    except:
        percentage_discount = 'per_dis_failed'
    try:
        item_url = article.find_all('a', {"class":"cept-tt thread-link linkPlain thread-title--list js-thread-title"})[0]['href']
    except:
        item_url = 'url_failed'
    try:
        date_tag = article.find_all('div', {"class":"size--all-s flex boxAlign-jc--all-fe boxAlign-ai--all-c flex--grow-1 overflow--hidden"})
    except:
        date_tag = 'date_failed'

    print(f"--------------------{num}-----------------")
    print(f"name: {name}")
    print(f"item_id: {item_id}")
    print(f"discount_price: {discount_price}")
    print(f"regular_price: {regular_price}")
    print(f"percentage_discount: {percentage_discount}")
    print(f"item_url: {item_url}")
    print(f"date_tag: {date_tag}")

    num += 1





--------------------0-----------------
name: Czerwona papryka kg @Lidl
item_id: 704489
discount_price: 4.99
regular_price: 9.99
percentage_discount: -50.0
item_url: https://www.pepper.pl/promocje/czerwona-papryka-kg-at-lidl-704489
date_tag: [<div class="size--all-s flex boxAlign-jc--all-fe boxAlign-ai--all-c flex--grow-1 overflow--hidden"><span class="text--b metaRibbon lbox--v-1 boxAlign-ai--all-c overflow--wrap-off space--l-3 text--color-blue"><svg class="icon icon--clock text--color-blue space--mr-1" height="22px" width="22px"><use xlink:href="/assets/img/ico_fff5e.svg#clock"></use></svg><span>Jutro</span></span><span class="hide--toW2 metaRibbon lbox--v-1 boxAlign-ai--all-c overflow--wrap-off space--l-3 text--color-greyShade"><svg class="icon icon--hourglass text--color-greyShade space--mr-1" height="22px" width="18px"><use xlink:href="/assets/img/ico_fff5e.svg#hourglass"></use></svg><span><span class="text--b">05/08/2023</span></span></span><span class="hide--toW2 hide--toW5 metaR

In [50]:
a = article.find_all(attrs={'class': "thread-price text--b cept-tp size--all-l size--fromW3-xl"})


if len(a) == 0:
    print("empty")
else:
    print("not empty")

a_pr

empty


In [31]:
print(article['id'].get_text())

KeyError: 'id'

In [10]:
print(imported_tag.prettify())

<article class="thread cept-thread-item thread--type-list imgFrame-container--scale thread--deal" data-handler="history thread-click" data-history='{"endpoint":"https://www.pepper.pl/nowe","replace":true,"data":{"scrollTo":"#thread_703221","offset":70,"scrollContainer":"#main"},"events":["history","click"],"delegate":true}' data-ocular='{"thread_ids":703221}' data-t="thread" data-t-d='{"id":703221}' data-t-view="" data-t-view-twig="" id="thread_703221">
 <div class="threadGrid thread-clickRoot">
  <div class="threadGrid-image space--r-3">
   <span class="imgFrame imgFrame--darken imgFrame--noBorder thread-listImgCell">
    <img alt="PARKSIDE Wiertarkowkrętarka akumulatorowa 20 V PABS 20-Li E6 (bez aku i ładowarki)" class="thread-image width--all-auto height--all-auto imgFrame-img" height="300" loading="lazy" src="https://static.pepper.pl/threads/raw/NA0Ha/703221_1/re/300x300/qt/60/703221_1.jpg" width="394"/>
   </span>
  </div>
  <div class="threadGrid-headerMeta">
   <div class="flex 

In [7]:
import signal
def signal_handler_for_continuously_scrapping(signal):
    global interrupted
    interrupted = True

In [9]:
interrupted = False

signal_handler_for_continuously_scrapping(signal)

print(interrupted)

True


In [2]:
from datetime import date, datetime, timezone

start_time = datetime.utcnow().replace(tzinfo=timezone.utc)

for i in range(1,1000):
    pass

end_time = datetime.utcnow().replace(tzinfo=timezone.utc)
action_execution_datetime = end_time - start_time

In [3]:
print(action_execution_datetime)

0:00:00.000059
